In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()
    

In [3]:
##membaca dataset
training = spark.read.csv('contoh1.csv', header=True,inferSchema=True)

In [5]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent")

In [9]:
output = featureassembler.transform(training)

In [10]:
output.show()

+---------+---+----------+------+-----------+
|     Name|age|Experience|Salary|Independent|
+---------+---+----------+------+-----------+
|    Krish| 31|        10| 30000|[31.0,10.0]|
|Sudhanshu| 30|         8| 25000| [30.0,8.0]|
|    Sunny| 29|         4| 20000| [29.0,4.0]|
|     Paul| 24|         3| 20000| [24.0,3.0]|
|   Harsha| 21|         1| 15000| [21.0,1.0]|
|  Shubham| 23|         2| 18000| [23.0,2.0]|
+---------+---+----------+------+-----------+



In [11]:
finalized_data = output.select("Independent", "Salary")

In [12]:
finalized_data.show()

+-----------+------+
|Independent|Salary|
+-----------+------+
|[31.0,10.0]| 30000|
| [30.0,8.0]| 25000|
| [29.0,4.0]| 20000|
| [24.0,3.0]| 20000|
| [21.0,1.0]| 15000|
| [23.0,2.0]| 18000|
+-----------+------+



In [32]:
from pyspark.ml.regression import LinearRegression
##train text split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent', labelCol='Salary')
regressor=regressor.fit(train_data)

In [33]:
###coefficients
regressor.coefficients

DenseVector([-102.53, 1688.6818])

In [34]:
###Intercepts
regressor.intercept

16470.03994673731

In [35]:
pred_results=regressor.evaluate(test_data)

In [36]:
###Prediction
pred_results.predictions.show()

+-----------+------+-----------------+
|Independent|Salary|       prediction|
+-----------+------+-----------------+
| [30.0,8.0]| 25000|26903.59520639148|
+-----------+------+-----------------+



In [37]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1903.595206391481, 3623674.709796625)